In [1]:
import pandas as pd
import folium

In [2]:
data = pd.read_csv("./data.csv")

In [3]:
data.head()

,Name,date_time,Year,Month,Day,Hours_in_UTC,Latitude,Longitude,Maximum_sustained_wind_in_knots
0,ABLE,1950/8/12 00:00,1950,8,12,0,17.1,-55.5,35
1,ABLE,1950/8/12 06:00,1950,8,12,6,17.7,-56.3,40
2,ABLE,1950/8/12 12:00,1950,8,12,12,18.2,-57.4,45
3,ABLE,1950/8/12 18:00,1950,8,12,18,19.0,-58.6,50
4,ABLE,1950/8/13 00:00,1950,8,13,0,20.0,-60.0,50


In [4]:
total_rows = len(data['Name'])
print(total_rows)

27106


In [5]:
'''
Visualized the historical hurricane on map based on its first original location
'''

# Sort dataframe by year, month, and day to get the earliest point for each hurricane
df_sorted = data.sort_values(by=['date_time'])

# Drop duplicate hurricanes, keeping only the earliest point for each one
df_earliest = df_sorted.drop_duplicates(subset='Name', keep='first')

# Create a map centered around the data's average location
m = folium.Map(location=[df_earliest['Latitude'].mean(), df_earliest['Longitude'].mean()], zoom_start=4)

# Add data points to the map
for idx, row in df_earliest.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Name']} ({row['Year']})",
        icon=folium.Icon(icon='cloud', color='blue')
    ).add_to(m)
m
# Save map to an HTML file
#m.save('hurricanes_map.html')


In [6]:
'''
Visualized the historical hurricane between 2017 to 2022 on the map and used different color to distribute the point
'''

# Filter data for years 2017 - 2022
df_filtered = data[(data['Year'] >= 2017) & (data['Year'] <= 2022)]

# Sort dataframe by year, month, and day to get the earliest point for each hurricane in the filtered range
df_sorted = df_filtered.sort_values(by=['date_time'])

# Drop duplicate hurricanes, keeping only the earliest point for each one
df_earliest = df_sorted.drop_duplicates(subset='Name', keep='first')

# Create a color map based on years
year_to_color = {year: color for year, color in zip(range(2017, 2023), 
                                                    ['red', 'blue', 'green', 'purple', 'orange', 'lightblue'])}

# Create a map centered around the data's average location
m = folium.Map(location=[df_earliest['Latitude'].mean(), df_earliest['Longitude'].mean()], zoom_start=4)

# Add data points to the map
for idx, row in df_earliest.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Name']} ({row['Year']})",
        icon=folium.Icon(icon='cloud', color=year_to_color[row['Year']])
    ).add_to(m)

legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 130px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background:white;
            ">
    &nbsp;<b>Year Legend</b><br>
'''
for year, color in year_to_color.items():
    legend_html += f'<i class="fa fa-circle" style="color:{color}"></i>&nbsp;{year}<br>'

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# Save map to an HTML file
#m.save('hurricanes_map_2017_2022.html')
m

In [8]:
'''
Map of Category 5 Hurricane
Created a map that only include historical hurricane from 2017 to 2022
'''

# Filter data for years 2017 - 2022 and where Maximum_sustained_wind_in_knots is greater than 137 (Category 5)
df_filtered = data[(data['Year'] >= 2017) & (data['Year'] <= 2022) & (data['Maximum_sustained_wind_in_knots'] >= 137)]

# Sort dataframe by year, month, and day
df_sorted = df_filtered.sort_values(by=['date_time'])
df_cat5 = df_sorted.drop_duplicates(subset='Name', keep='first')

# Create a color map based on years
year_to_color = {year: color for year, color in zip(range(2017, 2023),
                                                    ['red', 'blue', 'green', 'purple', 'orange', 'purple'])}
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]


# Create a map centered around the average location of the Category 5 hurricanes
m = folium.Map(location=[df_cat5['Latitude'].mean(), df_cat5['Longitude'].mean()], zoom_start=4)


for idx, row in df_cat5.iterrows():
    color = year_to_color.get(row['Year'], "gray")  # Default to "gray" if the year doesn't match any key
    month_name = months[int(row['Month']) - 1]  # Convert month number to month name
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Name']} ({row['Year']})",
        icon=folium.DivIcon(html=f"""<div style="color: {color};">{row['Name']} {int(row['Year'])} {month_name}</div>""")
    ).add_to(m)

# Save map to an HTML file
#m.save('category5_hurricanes_map_2017_2022.html')
m

In [9]:
# Detailed information of category 5 hurricane from 2017 to 2022
df_cat5

,Name,date_time,Year,Month,Day,Hours_in_UTC,Latitude,Longitude,Maximum_sustained_wind_in_knots
23884,MARIA,2017/9/19 00:00,2017,9,19,0,15.3,-61.1,145
23668,IRMA,2017/9/5 12:00,2017,9,5,12,16.7,-57.8,150
24581,MICHAEL,2018/10/10 17:30,2018,10,10,17,30.0,-85.5,140
24754,DORIAN,2019/9/1 06:00,2019,9,1,6,26.4,-75.6,145
24986,LORENZO,2019/9/29 03:00,2019,9,29,3,24.3,-45.0,140
26947,IAN,2022/9/28 12:00,2022,9,28,12,26.0,-82.7,140


In [10]:
'''
Research for average original location of hurrican happened in September and October between 1990 to 2022
'''

# Filter data for hurricanes between September and October
df_filtered = data[(data['Year'] > 1990)&(data['Month'] >= 9) & (data['Month'] <= 10) ]

df_sorted = df_filtered.sort_values(by=['date_time'])

# Drop duplicate hurricanes, keeping only the first point for each Category 5 hurricane in this time frame
df_filtered_1 = df_sorted.drop_duplicates(subset='Name', keep='first')

# Group by year and compute average latitude and longitude
grouped = df_filtered_1.groupby('Year').agg(average_latitude=('Latitude', 'mean'), average_longitude=('Longitude', 'mean')).reset_index()

'''
The data of EL Nino: https://ggweather.com/enso/oni.htm
'''
# List of EL Nino & La Nino
strong_el =[1957, 1958, 1965, 1966, 1972, 1973, 1987, 1988, 1982, 1983, 1991, 1992, 1997, 1998, 2015, 2016]
strong_la = [1973, 1973, 1975, 1976, 1988, 1989, 1998, 1999, 1999, 2000, 2007, 2008, 2010, 2011]

# Define color mapping
def get_color(year):
    if year in strong_el:
        return 'red'
    elif year in strong_la:
        return 'green'
    else:
        return 'blue'

def compute_avg_location_for_color(color):
    filtered_years = []
    
    if color == "red":
        filtered_years = strong_el
    elif color == "green":
        filtered_years = strong_la
    else:  # for blue or normal
        filtered_years = [year for year in grouped['Year'] if year not in strong_el and year not in strong_la]
    
    category_data = grouped[grouped['Year'].isin(filtered_years)]
    return category_data['average_latitude'].mean(), category_data['average_longitude'].mean()

# Compute average locations for red, green, and blue categories
avg_lat_red, avg_lon_red = compute_avg_location_for_color("red")
avg_lat_green, avg_lon_green = compute_avg_location_for_color("green")
avg_lat_blue, avg_lon_blue = compute_avg_location_for_color("blue")

print("Strong EL Avg location:", avg_lat_red, avg_lon_red)
print("Strong La Avg location:", avg_lat_green, avg_lon_green)
print("Normal Avg location:", avg_lat_blue, avg_lon_blue)

# Create a map centered around the average locations
m = folium.Map(location=[grouped['average_latitude'].mean(), grouped['average_longitude'].mean()], zoom_start=4)

# Add data points to the map
for idx, row in grouped.iterrows():
    color = get_color(row['Year'])
    folium.map.Marker(
        [row['average_latitude'], row['average_longitude']],
        icon=folium.DivIcon(html=f"""<div style="color: {color};">{int(row['Year'])}</div>""")
    ).add_to(m)

folium.Marker([avg_lat_red, avg_lon_red], icon=folium.Icon(color='red', icon='info-sign'), popup="Avg Strong El Niño").add_to(m)
folium.Marker([avg_lat_green, avg_lon_green], icon=folium.Icon(color='green', icon='info-sign'), popup="Avg Strong La Niña").add_to(m)
folium.Marker([avg_lat_blue, avg_lon_blue], icon=folium.Icon(color='blue', icon='info-sign'), popup="Avg Normal").add_to(m)

legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 90px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white;
            ">
    <b>Legend:</b><br>
    <i class="fa fa-square" style="color: red"></i> Strong El Niño<br>
    <i class="fa fa-square" style="color: green"></i> Strong La Niña<br>
    <i class="fa fa-square" style="color: blue"></i> Normal
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))
# Save map to an HTML file
#m.save('hurricanes_avg_location_map.html')
m

Strong EL Avg location: 24.200380952380954 -53.14857142857143
Strong La Avg location: 21.327692743764175 -60.54362244897959
Normal Avg location: 22.475901875901872 -56.736327561327556
